In [1]:
from torch.utils.data import DataLoader
from resnet import ResNet
from sklearn.model_selection import train_test_split
import numpy as np
from custom_dataset import FinalDataset
from custom_dataloader import Custom_Dataloader
import torch
from torch import nn

In [2]:
file_path= "C:\\Users\\nicki\\OneDrive\\Desktop\\uni_2\\Thesis\\filteredData.hdf5"
dataset = FinalDataset(file_path)
indices = np.arange(len(dataset))
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)
train_dataset = FinalDataset(file_path, indices=train_indices)
val_dataset = FinalDataset(file_path, indices=val_indices)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset.indices)
    print(f"size: {size}")
    print(len(dataloader))
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y.unsqueeze(1))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss_item = loss.item()
            current = batch * len(X)
            print(f"loss: {loss_item:>7f} [{current:>5d}/{size:>5d}]")
            
def test_loop(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset.indices)
    num_batches = len(dataloader.dataset.indices) // dataloader.batch_size
    print(f"size: {size}, num_batches: {num_batches}")
    
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y.unsqueeze(1)).item()  
            predicted = (pred > 0.5).float()
            correct += (predicted == y.unsqueeze(1)).type(torch.float).sum().item()  
    # Average loss and accuracy
    test_loss /= num_batches
    accuracy = (100 * correct) / size
    print(f"Test Error: \n Accuracy: {accuracy:>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(f"Correct: {correct}, Test Loss: {test_loss}, Num Batches: {num_batches}, Size: {size}")


In [5]:
loss_fn = nn.BCEWithLogitsLoss()
model = ResNet(12,24)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
epochs = 10

for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer, device)
    test_loop(val_loader, model, loss_fn, device)
    
print("Done!")
torch.save(model.state_dict(), "model.pth")

Using device: cuda
Epoch 1
-------------------------------
size: 294831
1152
loss: 0.486008 [    0/294831]
loss: 0.313262 [25600/294831]
loss: 0.313262 [51200/294831]
loss: 0.313262 [76800/294831]
loss: 0.313262 [102400/294831]
loss: 0.313262 [128000/294831]
loss: 0.321074 [153600/294831]
loss: 0.313262 [179200/294831]
loss: 0.313262 [204800/294831]
loss: 0.313262 [230400/294831]
loss: 0.317168 [256000/294831]
loss: 0.313262 [281600/294831]
size: 73708, num_batches: 287
Test Error: 
 Accuracy: 0.3%, Avg loss: 0.317035 

Correct: 197.0, Test Loss: 0.31703453061888026, Num Batches: 287, Size: 73708
Epoch 2
-------------------------------
size: 294831
1152
loss: 0.321074 [    0/294831]
loss: 0.313262 [25600/294831]
loss: 0.317168 [51200/294831]
loss: 0.317168 [76800/294831]
loss: 0.313262 [102400/294831]
loss: 0.317168 [128000/294831]
loss: 0.313262 [153600/294831]
loss: 0.321074 [179200/294831]
loss: 0.317168 [204800/294831]
loss: 0.313262 [230400/294831]
loss: 0.317168 [256000/294831]
l

KeyboardInterrupt: 